<a href="https://colab.research.google.com/github/Joab-S/SimulacaoEstocasticaUFC/blob/main/lista3_SimEst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import random as r
import simpy as sp
import numpy as np

In [10]:
N_CLIENTES = 0
N_ATENDENTES = 1
TAXA_CHEGADA = 2
TAXA_ATENDIMENTO = 3
LIMITE = 300

In [ ]:
def chegada(env):
  global n
  while True:
    yield env.timeout(np.random.exponential(TAXA_CHEGADA))
    n = n + 1
    print("Alguem chegou \t (n = ", n, " em t = ", str(env.now), ")")
    if (n == 1):
      env.process(atendimento(env))

In [ ]:
def atendimento(env):
  global n
  while (n > 0):
    yield env.timeout(np.random.exponential(TAXA_ATENDIMENTO))
    n = n - 1
    print("Alguem saiu   \t (n = ", n, " em t = ", str(env.now), ")")

In [ ]:
env = sp.Environment()
env.process(chegada(env))
if (n > 0):
  env.process(atendimento(env))
env.run(until=LIMITE)
print("Fim da simulação")